In [1]:
import pandas as pd
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
eans = pd.read_csv('data/database.csv')['ean'].to_list()

In [3]:
def scrap_Amazon_sales(ean):
    '''
    Toma el ean de un libro y devuelve su posición
    en distintos ranking de ventas de Amazon
    '''
    # Ajustes del driver
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    wait = WebDriverWait(driver, 10)                 
    driver.delete_all_cookies()

    # Acceso al artículo de Amazon
    amazon_searcher = "https://www.amazon.com/advanced-search/books"
    driver.get(amazon_searcher)

    search_isbn = driver.find_element(By.XPATH, '//*[(@id = "field-isbn")]')
    search_isbn.send_keys(str(ean))
    wait.until(EC.element_to_be_clickable(search_isbn)).submit()

    wait.until(EC.url_changes(amazon_searcher))

    wait.until(EC.element_to_be_clickable((By.XPATH,
                                          '//*[contains(concat( " ", @class, " " ), concat( " ", "s-line-clamp-2", " " ))]')
                                         )).click()

    # Extracción información ventas
    soup = BeautifulSoup(driver.page_source)
    try: 
        top_sales = soup.find("div", {"id": "detailBulletsWrapper_feature_div"}).find_all(
        "ul", class_= "a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list")[1].get_text()
    except:
        top_sales = None

    # Cierre del driver
    driver.quit()
    
    return (ean, top_sales)

In [4]:
%%capture
# Realizamos el raspado web
top_sales_by_ean = []
for ean in eans:
    try:
        top_sales_by_ean.append(scrap_Amazon_sales(ean))
    except:
        continue

In [5]:
# Tratamos y normalizamos los valores obtenidos
d_tops = dict()
for ean, top in top_sales_by_ean:
    if top != None and top != '   ':
        tops_libro = top.strip().split('nº')[1:]
        for i, t in enumerate(tops_libro):
            t = re.sub(r'\(Ver el Top 100.*\)', '', t)
            tops_libro[i] = t.strip()
        d_tops[ean] = tops_libro
    
df_amazon_tops = pd.DataFrame([d_tops.keys(), d_tops.values()]).transpose()
df_amazon_tops.columns = ['ean', 'tops_amazon']

# Mostramos primeras filas 
display(df_amazon_tops.head())

,ean,tops_amazon
0,9788433981066,"['3,322,967 en Tienda Kindle', '6,137 en Biogr..."
1,9788433981134,"['1,835,602 en Libros', '63,596 en Memorias (L..."
2,9788433981073,"['804,658 en Tienda Kindle', '641 en Biografía..."
3,9788433981103,"['377,300 en Tienda Kindle', '844 en Literatur..."
4,9788433981110,"['2,090,463 en Libros', '99,130 en Libros en e..."


In [6]:
# Añadimos la información obtenida al dataset original
df = pd.read_csv("data/database.csv")
df = pd.merge(df,df_amazon_tops,on='ean',how='left')

display(df)

# Y finalmente guardamos el dataset definitivo en un archivo csv
df.to_csv('data/coleccion_anagrama.csv', index=False)

,isbn,ean,código,título,autor,precio,páginas,fecha_publicación,colección,traducción,sinopsis,ebook,tops_amazon
0,978-84-339-8106-6,9788433981066,PN 1066,En verano,"Knausgård, Karl Ove",18.9 €,400.0,24/11/2021,Panorama De Narrativas,"Asunción Lorenzo, Kirsti Baggethun",La portentosa culminación del Cuarteto de las ...,1,"['3,322,967 en Tienda Kindle', '6,137 en Biogr..."
1,978-84-339-8113-4,9788433981134,PN 1073,Historia de los abuelos que no tuve,"Jablonka, Ivan",21.9 €,424.0,23/02/2022,Panorama De Narrativas,Agustina Blanco,"La historia de los abuelos del autor, muertos ...",0,"['1,835,602 en Libros', '63,596 en Memorias (L..."
2,978-84-339-8107-3,9788433981073,PN 1067,Desde dentro,"Amis, Martin",24.9 €,624.0,17/11/2021,Panorama De Narrativas,Jesús Zulaika Goicoechea,Un libro ambicioso y deslumbrante que mezcla v...,1,"['804,658 en Tienda Kindle', '641 en Biografía..."
3,978-84-339-8110-3,9788433981103,PN 1070,Sed,"Nothomb, Amélie",17.9 €,128.0,02/02/2022,Panorama De Narrativas,Sergi Pàmies,Una apasionante y nothombiana reelaboración de...,1,"['377,300 en Tienda Kindle', '844 en Literatur..."
4,978-84-339-8111-0,9788433981110,PN 1071,Nacido de ninguna mujer,"Bouysse, Franck",19.9 €,304.0,02/02/2022,Panorama De Narrativas,Rosa Alapont Calderaro,Un manuscrito. Una joven enfrentada a un desti...,1,"['2,090,463 en Libros', '99,130 en Libros en e..."
